In [2]:
import pandas as pd
import numpy as np

In [3]:
log = pd.read_csv('log.csv', header=None)
users = pd.read_csv('users.csv',sep='\t',encoding='KOI8-R')

In [196]:
# Теперь нужно сделать так, чтобы с колонками было удобно работать.
log.columns = ['user_id','time','bet','win']
users.columns = ['user_id','email','geo']

In [5]:
log['time'] = log['time'].str.replace('[','')

In [6]:
#  Посчитайте количество пропусков в столбце time.
log['time'].isna().sum()

15

<span style="color: orange; font-weight: bold; font-size:16pt">Удаление пропусков</span>

Пропуски можно удалять автоматически. Во многих случаях это правильно, так как данные с большим количеством пропусков часто не имеют смысла и не приносят никакой пользы. 

Удалять данные с пропусками можно с помощью метода ***dropna().***

Параметр axis в методе ***dropna()*** говорит методу, по какой оси удалять значения.  

Если нужно удалить строки, в которых встречается пропуск (NaN), следует указать axis=0.  Зачем это делать? Например, у нас из 1000 примеров данных про пользователей пропуски есть в пяти. Разумно их удалить, так как их количество пренебрежимо мало.

Если нужно удалить столбцы, в которых встречается пропуск (NaN), нужно указывать axis=1. Зачем? Иногда в одном конкретном столбце пропусков настолько много, что с ними просто не хочется возиться - смысла в них все равно почти нет. 

Еще один интересный параметр - ***subset***. Что он делает? Если передать в него список значений по одной оси (например, названия столбцов) и задать при этом в параметре axis другую ось (в нашем случае 0), то мы удалим те строки, для которых в данных столбцах находится пропуск. То же самое работает и наоборот: нужно поменять axis на 1 и вместо названий столбцов передавать индексы строк.

In [7]:
# Удалите все столбцы, где есть пропуски. Запишите в поле, сколько осталось столбцов в данных после этого.
log1 = log.copy()
log1.dropna(axis=1)

,user_id
0,Запись пользователя № - user_919
1,Запись пользователя № - user_973
2,Запись пользователя № - user_903
3,Запись пользователя № - user_954
4,Запись пользователя № - user_954
...,...
995,Запись пользователя № - user_984
996,#error
997,#error
998,#error


In [8]:
# Удалите все строки, где есть пропуски. Запишите в поле, сколько осталось строк в данных после этого.

log2 = log.copy()
log2.dropna(axis=0)

,user_id,time,bet,win
14,Запись пользователя № - user_917,2019-01-02 8:57:36,145732.0,1987653.0
29,Запись пользователя № - user_942,2019-01-04 13:59:42,1678321.0,9876543.0
151,Запись пользователя № - user_982,2019-01-16 21:54:22,100.0,4749.0
189,Запись пользователя № - user_964,2019-01-21 18:34:44,200.0,4667.0
205,Запись пользователя № - user_931,2019-01-22 5:26:59,300.0,4319.0
...,...,...,...,...
967,Запись пользователя № - user_975,2019-04-19 22:25:15,1000.0,6108.0
971,Запись пользователя № - user_912,2019-04-20 10:35:49,10554.0,31799.0
972,Запись пользователя № - user_926,2019-04-20 10:35:50,10354.0,30244.0
976,Запись пользователя № - user_970,2019-04-20 10:35:54,10354.0,30691.0


С данными в столбцах bet и win мы разберемся позже: пропуски в этих столбцах требуют особого подхода.

А сейчас:

если есть пропуски в столбце user_id - удалите столбец user_id,

если есть пропуски в столбце time - удалите столбец time.

Запишите в поле ответа, количество оставшихся столбцов в данных, после этих действий.

In [9]:
empty_data = log.isna().sum()[log.isna().sum() > 0].reset_index()
empty_data.columns = ['Column', 'Count']
display(empty_data)

,Column,Count
0,time,15
1,bet,515
2,win,862


In [10]:
log2 = log.copy()
log2 = log2.drop(['time'], axis=1)
log2

,user_id,bet,win
0,Запись пользователя № - user_919,NaN,NaN
1,Запись пользователя № - user_973,NaN,NaN
2,Запись пользователя № - user_903,NaN,NaN
3,Запись пользователя № - user_954,NaN,NaN
4,Запись пользователя № - user_954,NaN,NaN
...,...,...,...
995,Запись пользователя № - user_984,9754.0,NaN
996,#error,10054.0,29265.0
997,#error,10454.0,NaN
998,#error,1000.0,NaN


<span style="color: orange; font-weight: bold; font-size:16pt">Дубли</span>

Дубли - это повторяющиеся строки в данных. В сложных случаях строки могут быть практически одинаковые, но не совсем.

Самая частая причина очень банальна: дубли появляются из-за человеческих ошибок или невнимательности.

Например, при добавлении записи в систему вы случайно два раза нажали на кнопку "добавить". Если система позволяет иметь одинаковые записи, поздравляю - у вас в данных появились дубли.

Еще одна причина - слияние баз данных. Например, вы переносите телефоны из контактной книжки (физической, в которую заносили номера ручкой на бумагу, такие были популярны в прошлом) в телефон. Назвали в одном месте Сашу Сашей, а в другом Александром. Да, это тоже дубль, просто сам случай немного более сложный.

Сложным случаям типа последнего можно посвятить очень много времени, так что это материал для другого, более продвинутого курса по анализу данных. Скажу лишь, что в каждом конкретном случае подозрительные случаи нужно рассматривать отдельно, это требует кропотливой работы. 

Мы же рассмотрим простой случай, когда у вас в данных есть идентичные строки.

В pandas есть метод для удаления дублей (дубликатов) - **drop_duplicates()**. Он просто удаляет повторяющиеся строки.

У данного метода тоже есть параметр subset, в этом случае нужно передавать список содержащий названия столбцов.  

In [11]:
# Удалите дубли среди столбцов user_id и time. Запишите в поле ниже, сколько осталось строк после удаления дублей.
log1 = log[['user_id', 'time']].copy()
log1 = log1.drop_duplicates()
log1

,user_id,time
0,Запись пользователя № - user_919,2019-01-01 14:06:51
1,Запись пользователя № - user_973,2019-01-01 14:51:16
2,Запись пользователя № - user_903,2019-01-01 16:31:16
3,Запись пользователя № - user_954,2019-01-01 17:17:51
4,Запись пользователя № - user_954,2019-01-01 21:31:18
...,...,...
991,Запись пользователя № - user_965,2019-04-20 12:55:41
992,Запись пользователя № - user_967,2019-04-20 14:59:36
993,Запись пользователя № - user_973,2019-04-20 17:09:56
994,Запись пользователя № - user_977,2019-04-20 18:10:07


<span style="color: orange; font-weight: bold; font-size:16pt">Преобразование к datetime</span>

Для преобразования столбца с датой в виде текста в дату формата datetime (см. Модуль "С1. Работа с датами"), необходимо использовать метод to_datetime() в библиотеке pandas.

Внутрь метода нужно передать тот столбец, который требуется преобразовать.

In [12]:
#  Уберите лишний символ, преобразуйте признак time к datetime. После этого найдите наибольшую дату и выведите ее без времени.

log5 = log.copy()
log5 = log5.dropna(axis = 0)
log5['time'] = pd.to_datetime(log5['time'])
log5['time'].max()

Timestamp('2019-04-20 12:55:41')

In [13]:
logg = pd.read_csv("log.csv")  
logg = logg.dropna()  
logg.columns = ['user_id', 'time', 'bet', 'win']  
logg['time'] = logg['time'].apply(lambda x: x[1:])  
logg['time'] = pd.to_datetime(logg['time'])  
logg['time'] = logg.time.apply(lambda x: x.minute)
logg['time'].head() 

13     57
28     59
150    54
188    34
204    26
Name: time, dtype: int64

<span style="color: orange; font-weight: bold; font-size:16pt">Извлечение признаков времени</span>


**Извлечение признаков времени**

Для начала вспомним, что мы можем делать с datetime. Вот примеры атрибутов, по которым мы можем обращаться к данным объектам:

year: возвращает год
month: возвращает месяц
day: возвращает день
hour, minute, second - час, минута, секунда
dayofweek - день недели, от 0 до 6, где 0 - понедельник, 6 - воскресенье

Кроме них, есть и другие интересные атрибуты, советуем посмотреть здесь.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html

Ранее в курсе вы разбирали метод apply(). Он позволяет применить определенную функцию к каждому элементу в столбце.

В метод apply() можно передавать обычные и lambda-функции.

Например, если мы хотим получить столбец, в котором каждым значением будет год из другого столбца (это и есть feature engineering - создание новых признаков из старых), мы можем сделать следующее:

In [14]:
log1 = log.copy()
log1['time'] = pd.to_datetime(log1['time'])  
year_column = log1['time'].apply(lambda x: x.year) 

Библиотека pandas позволяет использовать аксессор dt для упрощения подобной работы:

In [15]:
year_colums = log1['time'].dt.year

Аксессор - это атрибут столбца, который хранит переменные типа Timestamp, то есть переменные, которые были строковым представлением времени, а затем изменены с помощью pd.to_datetime(). Если вы попытаетесь обратиться к dt у столбца, в котором лежит что-то отличное от времени, вы получите ошибку.
Чуть больше можно увидеть здесь (не забудьте посмотреть в исходный код).
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html


Вы можете пользоваться любым из предложенных выше способов. 


In [16]:
# Используйте оригинальные данные log.csv, столбец time.
# Найдите минуту, которая встречалась в данных чаще всего. Введите ответ в поле ниже.
log2 = log.copy()
log2['time'] = pd.to_datetime(log2['time']) 

minute = log2['time'].dt.minute
minute.value_counts()

36.0    25
31.0    23
6.0     22
50.0    22
14.0    21
29.0    21
12.0    21
5.0     21
40.0    21
18.0    21
43.0    21
53.0    21
58.0    20
57.0    20
45.0    20
48.0    20
2.0     20
21.0    19
56.0    19
27.0    19
9.0     18
54.0    18
47.0    18
20.0    18
34.0    18
35.0    18
3.0     18
33.0    18
51.0    17
10.0    17
55.0    16
25.0    16
17.0    16
49.0    15
28.0    15
24.0    15
30.0    15
38.0    15
15.0    15
37.0    15
16.0    15
7.0     14
4.0     14
44.0    14
19.0    14
32.0    14
59.0    13
41.0    13
1.0     13
0.0     12
23.0    12
39.0    12
22.0    11
13.0    11
42.0    11
52.0    10
46.0    10
8.0      9
26.0     9
11.0     6
Name: time, dtype: int64

In [17]:
# Найдите месяц, который встречался в данных реже всего. Введите ответ в поле ниже.
month = log2['time'].dt.month
month.value_counts(ascending = True)


4.0    170
2.0    259
3.0    264
1.0    292
Name: time, dtype: int64

In [18]:
# Посчитайте, сколько дней в данных являются выходными (то есть субботой или воскресеньем). Введите ответ в поле ниже.
#V1
weekdays = log2['time'].dt.weekday
a = weekdays[weekdays == 5].value_counts()
b = weekdays[weekdays == 6].value_counts()
print(a)
print(b)
152+131

5.0    152
Name: time, dtype: int64
6.0    131
Name: time, dtype: int64


283

In [19]:
# V2
log2['weekday'] = log2['time'].dt.weekday
#weekday_counts = log2.groupby('weekday').aggregate(sum)
log2[(log2['weekday'] == 5) | (log2['weekday'] == 6)].count()

user_id    283
time       283
bet        127
win         37
weekday    283
dtype: int64

In [21]:
#V3
log2 = log.copy()
log2['time'] = pd.to_datetime(log['time']) # convert to date
log2['time'].dt.weekday.apply(lambda x: x==5 or x==6).sum()

283

In [43]:
# Посчитайте, какое время дня встречается в данных реже всего. Введите ответ в поле ниже: ночь, утро, день или вечер.
# Договоримся, что с 0 до 5 часов - ночь, с 6 до 11 - утро, с 12 до 17 - день, с 18 до 23 - вечер.
# Подсказка: можно использовать value_counts(). Кроме этого, потребуется написать функцию, которая преобразует дату во время дня
log_d = log.copy()
#log_d = log_d.dropna(axis = 0)
log_d['time'] = pd.to_datetime(log_d['time'])
res = log_d['time'].dt.hour.value_counts().sort_values().reset_index()
# res сколько раз каждый час встречается в дата сете
print('Night', res.query('0 < index <= 5')['time'].sum())  # night
print('Morning', res.query('6 < index <= 11')['time'].sum())  # Morn
print('Day', res.query('12 < index <= 17')['time'].sum())  # Day
print('Evening', res.query('18 < index <= 23')['time'].sum())  # Even
display(res)

Night 228
Morning 206
Day 198
Evening 185


,index,time
0,7.0,32
1,19.0,33
2,16.0,33
3,11.0,34
4,21.0,35
5,15.0,37
6,20.0,37
7,0.0,37
8,9.0,39
9,23.0,39


Давайте повторим то, что мы прошли в этой секции. Напишите код, который создаст признак hour из признака time в датасете log.csv. Для этого:

1. загрузите датасет log.csv в переменную log, дальше работать будем с ней;

2. установите имена столбцов: ['user_id', 'time', 'bet', 'win'];

3. избавьтесь от пропусков в log;

4. приведите переменную time к подходящему для извлечения признаков виду;

5. получите значение часа для каждой строки в переменной time и запишите в столбец hour в log.

Результатом будет таблица log со столбцом hour внутри.

In [54]:
log = pd.read_csv('log.csv', header=None)
log.columns = ['user_id','time','bet','win']
log = log.dropna(axis=0)
log['time'] = log['time'].str.replace('[','')
log['time'] = pd.to_datetime(log['time'])
log['hour'] = log.time.dt.hour


,user_id,time,bet,win,hour
14,Запись пользователя № - user_917,2019-01-02 08:57:36,145732.0,1987653.0,8
29,Запись пользователя № - user_942,2019-01-04 13:59:42,1678321.0,9876543.0,13
151,Запись пользователя № - user_982,2019-01-16 21:54:22,100.0,4749.0,21
189,Запись пользователя № - user_964,2019-01-21 18:34:44,200.0,4667.0,18
205,Запись пользователя № - user_931,2019-01-22 05:26:59,300.0,4319.0,5
...,...,...,...,...,...
967,Запись пользователя № - user_975,2019-04-19 22:25:15,1000.0,6108.0,22
971,Запись пользователя № - user_912,2019-04-20 10:35:49,10554.0,31799.0,10
972,Запись пользователя № - user_926,2019-04-20 10:35:50,10354.0,30244.0,10
976,Запись пользователя № - user_970,2019-04-20 10:35:54,10354.0,30691.0,10


<span style="color: orange; font-weight: bold; font-size:16pt">Обработка пропусков</span>

Ранее мы говорили о пропусках - местах в данных, где по какой-то причине ничего нет.

Что мы делали с пропусками? Удаляли их.

На самом деле, удаление пропусков - довольно грубое решение, потому что мы можем случайно выбросить что-то полезное. Например, у вас есть данные о людях: пол, возраст, цвет глаз, город. При этом для большинства людей не записан цвет глаз. Удалять строки с пропусками может быть неудачным решением -  у нас почти не останется данных. Удалить столбец с цветом глаз - решение получше. Возможно, есть решение еще лучше - например, заполнить все пропуски цветом глаз "карие". Ответ на вопрос "Правильно ли будет так сделать?" можно получить после уточнения информации: в каких-то странах преобладает один цвет глаз, в каких-то - другой.

Поэтому довольно часто можно заполнить пропуски, сделав некоторые обоснованные предположения.

Сейчас мы познакомимся с самым простым способом заполнять пропуски - заполнением константой.

каждый пропуск в столбце мы заполним одним и тем же числом.

Почему мы вообще говорим о заполнении пропусков вместо того, чтобы просто удалить их? Потому что после удаления пропусков у нас останется слишком мало данных, а нам хочется получить какие-то инсайты из них.

<span style="color: orange; font-weight: bold; font-size:16pt">Заполнение константой</span>

Что такое константа? Это просто число.

Посмотрим на признак bet в наших данных (log.csv).

"Bet" означает "ставка".  Некоторые значения в данных заполнены цифрами - поэтому мы можем сделать предположение, что это сумма в рублях, т.е. ставка. 

Можно также предположить, что если в данных в этом месте пропуск - значит, человек не делал ставку. Другими словами, ставка в данном случае равна 0.

Чтобы заполнить пропуски в столбце каким-то значением, можно использовать метод *fillna()* у самого столбца. Аргументом этого метода будет число, которое появится на месте пропусков.

In [118]:
# Посчитайте, сколько раз люди приходили, но не делали ставок. Для этого заполните пропуски в столбце 
# bet значением 0 и посчитайте количество таких значений.

log = pd.read_csv('log.csv', header=None)
log.columns = ['user_id','time','bet','win']
log1 = log.copy()
log1['bet'] = log1['bet'].fillna(0)
log1['bet'].value_counts()

0.0          515
500.0         50
100.0         48
300.0         42
800.0         42
200.0         40
400.0         40
700.0         38
600.0         37
1000.0        37
900.0         30
9754.0        10
10554.0       10
10154.0        7
10254.0        7
10454.0        6
9954.0         6
10754.0        5
10654.0        5
10354.0        4
10054.0        4
9854.0         2
5000.0         1
27000.0        1
9876.0         1
156789.0       1
12945.0        1
12548.0        1
145732.0       1
1678321.0      1
98753.0        1
123981.0       1
104540.0       1
8700.0         1
950.0          1
8734.0         1
7650.0         1
Name: bet, dtype: int64

<span style="color: orange; font-weight: bold; font-size:16pt">Заполнение с помощью функции</span>

Теперь поработаем с признаком win, в котором тоже есть пропуски.

Иногда нужно заполнять пропуски не одним и тем же числом, а разными, в зависимости от какого-то условия. Перед нами именно этот случай.

Предположим, что если в признаке win находится пропуск, то выигрыша не было. Здесь два возможных случая:

Человек не делал ставки и ничего не выиграл. То есть просто пришел, посмотрел и ушел.
Человек делал ставку, но не выиграл. Значит, выигрыш на самом деле является отрицательным значением - это проигрыш.
Предлагаем вам написать метод, который заполнит пропуски в признаке win в соответствии с предположением выше. 

Для этого можно применить метод apply() ко всей таблице и передать ему функцию, которая вычисляет размер выигрыша (или проигрыша) по следующей схеме:

Если значение в столбце win существует (не пропуск) - вернуть это же значение. Это значит, что человек выиграл.
Если вместо значения в столбце win пропуск, вернуть 0.
На выходе получится столбец без пропусков. Следующим шагом будет замена старого столбца win на новый.

In [119]:
def fillna_win(row):  
    if row.isna().win == True:
        return 0
    return row.loc['win'] 
    
new_win = log1.apply(lambda row: fillna_win(row), axis=1)  

# Заменяем старый столбец с пропусками на новый без пропусков  
log1['win'] = new_win 
log1

,user_id,time,bet,win
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,0.0,0.0
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,0.0,0.0
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,0.0,0.0
3,Запись пользователя № - user_954,[2019-01-01 17:17:51,0.0,0.0
4,Запись пользователя № - user_954,[2019-01-01 21:31:18,0.0,0.0
...,...,...,...,...
995,Запись пользователя № - user_984,[2019-04-20 9:59:58,9754.0,0.0
996,#error,NaN,10054.0,29265.0
997,#error,NaN,10454.0,0.0
998,#error,NaN,1000.0,0.0


In [122]:
# Посчитайте, сколько раз участники ставок проиграли деньги. То есть посчитайте количество строк, 
# для которых в столбце win находится отрицательное значение.
def fillna_win(row):  
    if row.win == 0:
        return 0 - row.loc['bet']
    return row.loc['win'] 
    
new_win = log1.apply(lambda row: fillna_win(row), axis=1)  

# Заменяем старый столбец с пропусками на новый без пропусков  
log1['win'] = new_win 
log1['win'].apply(lambda x: x<0).sum()


347

Используйте модифицированный в прошлой секции датасет log.csv. Результат запишите числом в поле ниже.

Подсказка: можно использовать sum().
Создайте признак net, хранящий сумму выигрыша с учетом ставки. Для этого используйте следующий алгоритм:
- если значение признака win меньше 0 - присвоить значение признака win признаку net;

- во всех остальных случаях - из значения признака win вычтите значение признака bet и полученное значение присвойте признаку net.

После этого посчитайте, у скольких людей выигрыш положительный.

In [126]:
def net_net(row):
    if row.win < 0:
        return row.loc['win']
    return row.loc['win'] - row.loc['bet']
log1['net'] = log1.apply(lambda row: net_net(row),axis=1)
log1['net'].apply(lambda x: x>0).sum()

138

Используйте датасет log.csv, получившийся в результате выполнения предыдущего задания. Посчитайте среднее значение выигрыша (из столбца net) в тех случаях, когда выигрыш больше 0. Результат округлите до целого, отбросив дробную часть.

Подсказка: можно использовать mean().

In [129]:
log1[log1['net']>0]['net'].mean()

80253.33333333333

Посчитайте медианное значение выигрыша (из столбца net) в тех случаях, когда выигрыш больше 0. Результат округлите до целого, отбросив дробную часть.

In [130]:
log1[log1['net']>0]['net'].median()

5347.0

Как можно посчитать среднее значение для столбца bet, не учитывая при подсчете пропуски?

In [132]:
log.bet.mean()
#log.bet.sum() / log.bet.dropna().shape[0]
#log['bet'].dropna().mean()
#np.mean(log.bet)

6785.738144329897

Используйте модифицированный исходный датасет log.csv.

При модификации датасета log.csv, пропущенные значения в столбцах bet и win замените на 0, cоздайте столбец net, хранящий сумму выигрыша с учетом ставки (для этого из столбца win поэлементно вычтите столбец bet и запишите в новый столбец).

Посчитайте, какой процент посещений букмекерской конторы оборачивался ставкой. Для этого поделите количество ставок (значений больше 0) на общее количество посещений конторы. Результат округлите до одного знака после запятой.

In [169]:
round(len(log1[log1['bet']>0]['win'])/len(log1),3)*100

48.5

Используйте датасет log.csv, получившийся в результате модификации при выполнении первого задания этого блока.

Посчитайте среднее значение ставки (из столбца bet) в тех случаях, когда ставка была сделана. Запишите результат, отбросив дробную часть.
Подсказка: можно использовать mean().

In [170]:
log1[log1['bet']>0]['bet'].mean()


6785.738144329897

Используйте датасет log.csv, получившийся в результате модификации при выполнении первого задания этого блока.

Посчитайте средний выигрыш (из столбца net) в тех случаях, когда ставка была сделана. В ответ запишите полученное число с отброшенной дробной частью.
Пояснение: выигрыш в данном случае означает изменение количества денег и может быть отрицательным. В таком случае это проигрыш.

Подсказка: можно использовать mean().

In [176]:
log1[(log1['bet']>0)]['net'].mean()

20421.892783505155

Посчитайте среднее значение потерь при проигрыше (из столбца net). Результат округлите до целого, отбросив дробную часть

In [180]:
log1[(log1['bet']>0)&(log1['net']<0)]['net'].mean()

-3372.743515850144

Посчитайте, какой процент ставок заканчивается выигрышем, а какой - проигрышем. Сравните эти значения и ответьте, какое из них больше.

In [187]:
print(len(log1[(log1['bet']>0)&(log1['net']<0)])/len(log1)*100) # Lose
print(len(log1[(log1['bet']>0)&(log1['net']>0)])/len(log1)*100) # win

34.699999999999996
13.8


Давайте повторим то, что мы прошли в этой секции. Напишите код, который узнает, чему была равна минимальная ставка и сколько людей сделали такую ставку. Для этого:

1. загрузите датасет log.csv;

2. посчитайте, чему равна минимальная ставка;

3. посчитайте, сколько раз была сделана минимальная ставка, и запишите результат в переменную min_bet_amount в виде целого числа.

In [194]:
log = pd.read_csv('log.csv', header=None)
log.columns = ['user_id','time','bet','win']
log['bet'].min()
min_bet_amount = log[log['bet'] == log['bet'].min()]['bet'].count()

<span style="color: orange; font-weight: bold; font-size:16pt">Повторение merge/groupby</span>

Повторим часть предобработки, которую мы должны были выполнить ранее:


In [218]:
users.user_id = users.user_id.apply(lambda x: x.lower()) 

log1 = lo1g[log1.user_id != '#error']    
log1.user_id = log1.user_id.str.split(' - ').apply(lambda x: x[1])

,user_id,time,bet,win,net
0,user_919,[2019-01-01 14:06:51,0.0,0.0,0.0
1,user_973,[2019-01-01 14:51:16,0.0,0.0,0.0
2,user_903,[2019-01-01 16:31:16,0.0,0.0,0.0
3,user_954,[2019-01-01 17:17:51,0.0,0.0,0.0
4,user_954,[2019-01-01 21:31:18,0.0,0.0,0.0
...,...,...,...,...,...
991,user_965,[2019-04-20 12:55:41,800.0,6927.0,6127.0
992,user_967,[2019-04-20 14:59:36,10154.0,-10154.0,-10154.0
993,user_973,[2019-04-20 17:09:56,10254.0,-10254.0,-10254.0
994,user_977,[2019-04-20 18:10:07,10354.0,-10354.0,-10354.0


Tеперь объединим данные с помощью метода pd.merge():

In [229]:
df = pd.merge(log1, users, on='user_id')
df

,user_id,time,bet,win,net,email,geo
0,user_919,[2019-01-01 14:06:51,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
1,user_919,[2019-01-30 10:06:00,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
2,user_919,[2019-02-05 14:33:44,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
3,user_919,[2019-02-14 11:38:05,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
4,user_919,[2019-03-02 4:23:36,300.0,-300.0,-300.0,Chikkaverle@icloud.com,Хабаровск
...,...,...,...,...,...,...,...
970,user_932,[2019-02-24 22:40:06,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск
971,user_932,[2019-03-15 10:56:14,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск
972,user_932,[2019-03-18 10:13:24,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск
973,user_932,[2019-03-27 12:18:24,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск


**Groupby**

In [233]:
df.groupby('user_id').win.median().median()

0.0

В данном случае мы группируем данные по признаку user_id.

После этого мы в каждой группе выбираем признак win.

Затем мы берем медиану каждой группы по признаку win и на выходе получаем таблицу, в которой индексом является признак user_id. В этой таблице единственный столбец - медиана по каждой группе (то есть по каждому пользователю).

Наконец, последний вызов median() дает нам медиану по предыдущему столбцу, то есть возвращает одно число.

Используйте датасет, который получился в результате всех преобразований выше (в том числе, заполнение пропусков). Ответ запишите в поле ниже в виде целого числа (отбросьте дробную часть).

Посчитайте медиану баланса по каждому пользователю. Для этого сгруппируйте по пользователям, возьмите признак net, просуммируйте по каждому пользователю и получите медиану.

In [237]:
df.groupby('user_id').net.sum().median()

1986.0

Сколько раз в среднем каждый человек приходит, не делая ставок, при условии, что у этого человека все-таки есть хотя бы одна ставка? Например: Человек посетил букмекерскую контору 5 раз из них 1 раз сделал ставку, 4 раза нет - условие выполняется. Человек посетил букмекерскую контору 5 раз из них ни разу ставку не сделал - условие не выполняется. Для того, чтобы узнать это, просуммируйте в каждой группе количество записей со ставкой, равной 0, и поделите на общее количество групп. Если при этом в группе нет записей со ставкой больше 0, считаем количество записей в данной группе равным 0.

In [275]:

mid = df.groupby('user_id')['bet'].sum().reset_index()
#df[df['user_id'] == 'user_942']
#df[df['bet']==0]['user_id'].value_counts()
mid.sort_values('bet')


,user_id,bet
2,user_901,600.0
5,user_904,600.0
33,user_932,700.0
37,user_936,800.0
91,user_991,800.0
...,...,...
87,user_987,46128.0
6,user_905,53395.0
11,user_910,232121.0
18,user_917,304221.0


,user_id,bet
2,user_901,600.0
5,user_904,600.0
33,user_932,700.0
37,user_936,800.0
91,user_991,800.0
...,...,...
87,user_987,46128.0
6,user_905,53395.0
11,user_910,232121.0
18,user_917,304221.0
